<a href="https://colab.research.google.com/github/phucle103198/Learning-Free-JAR/blob/main/FreeJAR_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import tensorflow as tf
import tensorflow_hub as hub
from keras.utils import to_categorical
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

In [5]:
data = pd.read_csv("./cheese.csv", encoding='latin-1',delimiter=';')
data

,ID_juge,Product,Hedonic_category,FreeJAR_description,Liking,Too_much,Not_enough,JAR
0,33002,CE,I like it a lot,I like the taste,8,0,0,0
1,33002,M,I like it a bit,I like this product but the taste is too strong.,"7,206575879",1,0,0
2,33002,B,I like it a bit,I like it but the taste is a bit strong.,"6,534021533",0,0,0
3,33002,R,I like it a bit,I like it but the texture is a little too soft.,"7,190606657",1,0,0
4,33002,C,I like it a lot,I like the taste but the texture is a bit stra...,7,0,0,0
...,...,...,...,...,...,...,...,...
611,33153,CE,I like it a lot,Appreciable I like very much,6,0,0,0
612,33153,S,I like it a lot,My favorite I like very much,6,0,0,0
613,33153,M,I like it a lot,Lack of energy I like very much,6,0,1,0
614,33153,E,I like it a lot,Farmer's milk I like very much,7,0,0,0


In [6]:
descriptions = data['FreeJAR_description'].fillna("").astype(str).tolist()
categories = data['Hedonic_category']

In [7]:
# Encode target labels
label_encoder = LabelEncoder()
categories_encoded = label_encoder.fit_transform(categories)
num_classes = len(label_encoder.classes_)

In [8]:
train_texts = descriptions
test_texts = descriptions
train_labels = categories_encoded
test_labels = categories_encoded

In [ ]:
# Train-test split
train_texts, test_texts, train_labels, test_labels = train_test_split(
    descriptions, categories_encoded, test_size=0.2, random_state=42
)

In [9]:
# Initialize tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [10]:
# Create custom dataset
class HedonicDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=self.max_len,
            return_tensors="pt"
        )
        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "label": torch.tensor(label, dtype=torch.long)
        }

In [11]:
# Create data loaders
batch_size = 16
train_dataset = HedonicDataset(train_texts, train_labels, tokenizer)
test_dataset = HedonicDataset(test_texts, test_labels, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Define model
class ClassificationModel(nn.Module):
    def __init__(self, num_classes):
        super(ClassificationModel, self).__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.fc1 = nn.Linear(self.bert.config.hidden_size, 64)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.fc2 = nn.Linear(64, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output  # [CLS] token representation
        x = self.fc1(pooled_output)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Initialize model, loss, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ClassificationModel(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

# Training loop
epochs = 3
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader)}")




model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Epoch 1/3, Loss: 0.946146398018568
Epoch 2/3, Loss: 0.6286296019187341
Epoch 3/3, Loss: 0.4147239243372893


In [12]:
# Evaluation
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        outputs = model(input_ids, attention_mask)
        _, preds = torch.max(outputs, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Classification report
print(classification_report(all_labels, all_preds, target_names=label_encoder.classes_))

                 precision    recall  f1-score   support

I don't like it       0.93      0.98      0.96       132
I like it a bit       0.97      0.90      0.94       239
I like it a lot       0.95      0.98      0.96       245

       accuracy                           0.95       616
      macro avg       0.95      0.96      0.95       616
   weighted avg       0.95      0.95      0.95       616



In [13]:
# Prediction on new data
def predict(text):
    model.eval()
    encoding = tokenizer(
        text, padding="max_length", truncation=True, max_length=128, return_tensors="pt"
    )
    input_ids = encoding["input_ids"].to(device)
    attention_mask = encoding["attention_mask"].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask)
        _, pred = torch.max(outputs, dim=1)
    return label_encoder.inverse_transform(pred.cpu().numpy())[0]

In [22]:
example_text = "This texture of product is little too soft!"
print("Predicted Hedonic Category:", predict(example_text))

Predicted Hedonic Category: I like it a lot
